<a href="https://colab.research.google.com/github/Madhuvod/Enhanced-Emotion-Recognition-using-V-MoE-/blob/main/MELD_Dataset_Video-to-frames.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import csv
import cv2
from pathlib import Path
from tqdm import tqdm
from google.colab import drive
import pandas as pd

In [2]:
drive.mount('/content/drive')

Mounted at /content/drive


/content/drive/MyDrive/MELD_Dataset/train_splits THIS IS THE FILE PATH OF TRAINING DATA


In [3]:
# Cell 3: Frame Extraction Function
def extract_frames(video_path, output_dir, utterance_id, emotion):
    """
    Extract frames from video and organize them by emotion and utterance.

    Parameters:
    - video_path: path to the video file (e.g., 'dev/dia0_utt8.mp4')
    - output_dir: base directory for saving frames (e.g., 'dataset/dev')
    - utterance_id: ID of the utterance (e.g., '0_8')
    - emotion: emotion label from CSV (e.g., 'angry')
    """
    # Step 1: Create emotion directory
    emotion_dir = os.path.join(output_dir, emotion.lower())
    os.makedirs(emotion_dir, exist_ok=True)

    # Step 2: Create utterance directory
    utterance_dir = os.path.join(emotion_dir, f"utterance_ID_{utterance_id}")
    os.makedirs(utterance_dir, exist_ok=True)

    # Step 3: Open and validate video file
    video = cv2.VideoCapture(video_path)
    if not video.isOpened():
        print(f"Error opening video: {video_path}")
        return None, 0

    # Step 4: Extract frames
    frame_count = 0
    success, frame = video.read()

    while success:
        # Save each frame as a JPG file
        frame_path = os.path.join(utterance_dir, f"frame_{frame_count:03d}.jpg")
        cv2.imwrite(frame_path, frame)
        frame_count += 1
        success, frame = video.read()

    # Step 5: Cleanup
    video.release()
    return utterance_dir, frame_count

In [5]:
import pandas as pd
data = pd.read_csv('/content/drive/MyDrive/MELD_Dataset/train_sent_emo.csv')
from google.colab import sheets
# sheet = sheets.InteractiveSheet(df=data)

In [6]:
num_rows = data.shape[0]
num_rows

9989

In [8]:
import os

folder_path = '/content/drive/MyDrive/MELD_Dataset/train_splits'

# Count video files (assuming they have .mp4 extension)
video_count = len([f for f in os.listdir(folder_path) if f.endswith('.mp4')])

print(f"Number of video files in the folder: {video_count}")

Number of video files in the folder: 4927


In [9]:
# Cell 4: Helper Function to Parse Filenames
import re
def get_utterance_from_filename(filename):
    """Extract dialogue and utterance numbers from filename."""
    pattern = r'dia(\d+)_utt(\d+)\.mp4'
    match = re.match(pattern, filename)
    if match:
        dialogue_num = match.group(1)
        utterance_num = match.group(2)
        return f"{dialogue_num}_{utterance_num}"
    return None

In [10]:
# Cell 5: Main Processing Function
def process_dataset(base_dir, csv_path, video_folder, output_folder):
    """Process all videos in a dataset split."""
    # Setup paths
    video_dir = os.path.join(base_dir, video_folder)
    frames_output_dir = os.path.join(base_dir, output_folder)
    new_csv_path = os.path.join(base_dir, f"{output_folder.split('/')[-1]}_processed.csv")

    # Create output directory
    os.makedirs(frames_output_dir, exist_ok=True)

    # Read CSV file and create emotion lookup dictionary
    df = pd.read_csv(csv_path)
    utterance_to_emotion = {}
    for index, row in df.iterrows():
        utterance_to_emotion[f"{row['Dialogue_ID']}_{row['Utterance_ID']}"] = row['Emotion']

    # Process videos and create new CSV
    with open(new_csv_path, 'w', newline='', encoding='utf-8') as new_csv:
        fieldnames = ['Utterance_ID', 'Emotion', 'frame_path', 'num_frames']
        writer = csv.DictWriter(new_csv, fieldnames=fieldnames)
        writer.writeheader()

        # Process each video file
        video_files = [f for f in os.listdir(video_dir) if f.endswith('.mp4')]
        for video_file in tqdm(video_files, desc=f"Processing {output_folder} videos"):
            # Get video path
            video_path = os.path.join(video_dir, video_file)

            # Extract utterance_id from filename
            utterance_id = get_utterance_from_filename(video_file)

            if not utterance_id:
                print(f"Warning: Could not parse filename: {video_file}")
                continue

            # Get emotion from CSV mapping
            emotion = utterance_to_emotion.get(utterance_id)

            if not emotion:
                print(f"Warning: No emotion found for utterance ID: {utterance_id}")
                continue

            # Extract frames and get info
            frame_dir, num_frames = extract_frames(video_path, frames_output_dir,
                                                 utterance_id, emotion)

            if frame_dir and num_frames > 0:
                writer.writerow({
                    'Utterance_ID': utterance_id,
                    'Emotion': emotion,
                    'frame_path': frame_dir,
                    'num_frames': num_frames
                })

In [ ]:
# Cell 6: Run the Processing
BASE_DIR = "/content/drive/MyDrive/MELD_Dataset"  # Update this path

DATASET_MAPPING = [
    {"csv_path": os.path.join(BASE_DIR, "test_sent_emo.csv"),
     "video_folder": "output_repeated_splits_test",
     "output_folder": "dataset/test"},
]

for dataset in DATASET_MAPPING:
    print(f"Processing {dataset['output_folder']} split...")
    process_dataset(
        BASE_DIR,
        dataset["csv_path"],
        dataset["video_folder"],
        dataset["output_folder"]
    )